In [ ]:
pip install google.genai

In [ ]:
# Works only for pdf 
from google import genai
from google.genai import types
import pathlib
import os
from dotenv import load_dotenv
load_dotenv()


client = genai.Client(api_key=os.environ.get("GEN_AI_API_KEY"))

filepath = pathlib. Path ( r"C:\Users\akash\Pictures\Screenshots\{E87A65C1-E5C0-402F-8329-FA79EB6685ED}.png")
prompt = "What is the document all about?"
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        types . Part. from_bytes (
            data=filepath.read_bytes(),
            mime_type="application/pdf",
        ),
        prompt,
    ],
)
print (response. text)


In [ ]:
# This works only for image
from google import genai
import os
from dotenv import load_dotenv
load_dotenv()

client = genai.Client(api_key=os.environ.get("GEN_AI_API_KEY"))

my_file = client.files.upload(file=r"C:\Users\akash\Pictures\Screenshots\Screenshot (1).png")

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[my_file, "Caption this image."],
)

print(response.text)

In [ ]:
# Working for both pdf as well as image 
import google.generativeai as genai
import mimetypes
from PIL import Image
import os
import textwrap
import pypdf 
import textwrap
from dotenv import load_dotenv
load_dotenv()


try:
    genai.configure(api_key=os.environ.get("GEN_AI_API_KEY"))
except KeyError:
    print("Error: GEMINI_API_KEY environment variable not set.")
    exit()

def generate_universal_caption(file_path: str) -> str:
   
    try:
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        mime_type, _ = mimetypes.guess_type(file_path)
        if mime_type is None:
            return f"Error: Could not determine the file type for {file_path}"
        
        main_type = mime_type.split('/')[0]

        if main_type == 'image':
            print(f"✅ Detected image file: {file_path}")
            prompt = "Generate a short, descriptive caption for this image also detect the department to which it should belong.."
            img = Image.open(file_path)
            response = model.generate_content([prompt, img])
            return response.text

        elif main_type == 'text':
            print(f"✅ Detected text document: {file_path}")
            prompt = "Summarize this document into a single, concise caption."
            with open(file_path, 'r', encoding='utf-8') as f:
                document_text = f.read()
            if not document_text.strip():
                 return "Error: The text file is empty."
            response = model.generate_content([prompt, document_text])
            return response.text

       
        elif mime_type == 'application/pdf':
            print(f"✅ Detected PDF file: {file_path}")
            prompt = "Summarize this document into a single, concise caption also detect the department to which it should belong."
            pdf_text = ""
            reader = pypdf.PdfReader(file_path)
            for page in reader.pages:
                pdf_text += page.extract_text()
            
            if not pdf_text.strip():
                return "Error: Could not extract any text from this PDF. It might be an image-only PDF."

            response = model.generate_content([prompt, pdf_text])
            return response.text
        
        else:
            return f"Error: Unsupported file type '{mime_type}'. This function supports images, .txt, and .pdf files."

    except FileNotFoundError:
        return f"Error: The file was not found at '{file_path}'."
    except Exception as e:
        return f"An unexpected error occurred: {e}"

caption=generate_universal_caption(r"C:\Users\akash\Downloads\KMRL-Recruitment-2023-Station-Controller-Posts-Apply-Now.jpg")
print(f"Caption:\n{textwrap.fill(caption, width=80)}")